In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import defaultCityDistricts, mkdir

import classroom
import teachers
import course
import aClass
import students
import meeting
import exams
import surveys
import attendance


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#This cell generates data

df_classrooms = classroom.generate(357, 999)
df_teachers = teachers.generate(300)
df_courses = course.generate(300, df_teachers)
df_classes, df_courses = aClass.generate(10, df_teachers, df_courses)
df_students = students.generate(800, df_classes)
# may enter infinite loop; if it happens restart the kernel and run the code again
df_meetings = meeting.generate(0, df_courses, df_classrooms, df_classes, "2022-05-25", "2022-06-03")
df_exams = exams.generate(0, df_meetings, df_students)
df_surveys = surveys.generate(0, df_meetings, df_students, df_courses)
df_attendance = attendance.generate(0, df_meetings, df_students, df_teachers, df_courses)

Generating schedule for 8 days between 2022-05-25 and 2022-06-03
Starting at 2024-04-09 11:32:19.457629


In [ ]:
#This cell loads data from csv files

path = "data"

df_classrooms = pd.read_csv(f"{path}/classrooms.csv")
df_teachers = pd.read_csv(f"{path}/teachers.csv")
df_courses = pd.read_csv(f"{path}/courses.csv")
df_classes = pd.read_csv(f"{path}/classes.csv")
df_students = pd.read_csv(f"{path}/students.csv")
df_meetings = pd.read_csv(f"{path}/meetings.csv")
df_exams = pd.read_csv(f"{path}/exams.csv")
df_surveys = pd.read_csv(f"{path}/surveys.csv")
df_attendance = pd.read_csv(f"{path}/attendance.csv")

In [4]:
#This cell purges unused rows
size =len(df_classrooms)
df_classrooms = df_classrooms[df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
df_classrooms_removed = df_classrooms[~df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
print(f"Removed {size - len(df_classrooms)} classrooms")

size =len(df_teachers)
df_teachers = df_teachers[df_teachers["tID"].isin(pd.concat([df_classes["tID"], df_courses["tID"]]).unique())]
df_teachers_removed = df_teachers[~df_teachers["tID"].isin(df_classes["tID"] + df_courses["tID"])]
print(f"Removed {size - len(df_teachers)} teachers")

size =len(df_courses)
df_courses = df_courses[df_courses["cID"].isin(df_classes["courses"].explode().unique())]
df_courses_removed = df_courses[~df_courses["cID"].isin(df_classes["courses"].explode().unique())]
print(f"Removed {size - len(df_courses)} courses")

size =len(df_classes)
df_classes = df_classes[df_classes["ClassName"].isin(df_students["ClassName"])]
df_courses_classes = df_classes[~df_classes["ClassName"].isin(df_students["ClassName"])]
print(f"Removed {size - len(df_classes)} classes")

Removed 47 classrooms
Removed 91 teachers
Removed 85 courses
Removed 0 classes


In [9]:
path = "data"
headers = True
indexing = False

mkdir(path)

df_students.drop(columns=["iq"]).to_csv(f"{path}/students.csv", index=indexing, header=headers)
df_classes.drop(columns=["Year", "courses"]).to_csv(f"{path}/classes.csv", index=indexing, header=headers)
df_teachers.to_csv(f"{path}/teachers.csv", index=indexing, header=headers)
df_courses.to_csv(f"{path}/courses.csv", index=indexing, header=headers)
df_classrooms.to_csv(f"{path}/classrooms.csv", index=indexing, header=headers)
df_meetings.drop(columns=["StartHour", "EndHour", "Year", "Month"]).to_csv(f"{path}/meetings.csv", index=indexing, header=headers)
df_surveys.to_csv(f"{path}/surveys.csv", index=indexing, header=headers)
df_exams.to_csv(f"{path}/exams.csv", index=indexing, header=headers)
df_attendance.to_csv(f"{path}/attendance.csv", index=indexing, header=headers)


Directory data already exists


In [ ]:
path = "data"
headers = True
indexing = False

mkdir(path)

df_students.to_csv(f"{path}/students.csv", index=indexing, header=headers)
df_classes.to_csv(f"{path}/classes.csv", index=indexing, header=headers)
df_teachers.to_csv(f"{path}/teachers.csv", index=indexing, header=headers)
df_courses.to_csv(f"{path}/courses.csv", index=indexing, header=headers)
df_classrooms.to_csv(f"{path}/classrooms.csv", index=indexing, header=headers)
df_meetings.to_csv(f"{path}/meetings.csv", index=indexing, header=headers)
df_surveys.to_csv(f"{path}/surveys.csv", index=indexing, header=headers)
df_exams.to_csv(f"{path}/exams.csv", index=indexing, header=headers)
df_attendance.to_csv(f"{path}/attendance.csv", index=indexing, header=headers)

In [11]:
path = "updates"

mkdir(path)

df = df_teachers.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Teachers SET CityDistrict = '{x['CityDistrict2']}' WHERE tID = {x['tID']};", axis=1)
df["update"].to_csv(f"{path}/updateTeachersCity.sql", index=False, header=False)

df = df_students.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET CityDistrict = '{x['CityDistrict2']}' WHERE sID = {x['sID']};", axis=1)
df["update"].to_csv(f"{path}/updateStudentsCity.sql", index=False, header=False)

df = df_students.sample(n=15)
df["ClassName2"] = df["ClassName"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET ClassName = '{x['ClassName']}' WHERE sID = {x['sID']};", axis=1)
df["update"].to_csv(f"{path}/updateStudentsClass.sql", index=False, header=False)

df = df_classes.sample(n=15)
df["tID2"] = df["tID"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Class SET tID = '{x['tID']}' WHERE ClassName = '{x['ClassName']}';", axis=1)
df["update"].to_csv(f"{path}/updateClassesTeacher.sql", index=False, header=False)

OSError: Cannot save file into a non-existent directory: 'updates'